In [1]:
# Imports and set torch device
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt
import kaolin as kal
import cv2
from collections import defaultdict
import trimesh
from scipy.spatial.distance import cdist
import torch
import torch.nn.functional as F

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

print('Torch will run on:', device)

object = 'bookshelf' 
obj_path = 'data/demo/' + object + '.obj'

Torch will run on: cuda:0


In [2]:
# Read mesh
mesh = kal.io.obj.import_mesh(
    obj_path,
    with_normals=True,
    with_materials=False,
)

vertices_tensor = mesh.vertices.to(device)
faces_tensor = mesh.faces.to(device)

vertices = vertices_tensor.detach().cpu().numpy()
faces = faces_tensor.detach().cpu().numpy()
colors =  mesh.vertex_normals.cpu().numpy()

print('Number of vertices: ', vertices.shape[0])
print('Number of faces: ', faces.shape[0])

Number of vertices:  5786
Number of faces:  8624


In [3]:
# Visualize mesh
trimeshMesh = trimesh.Trimesh(vertices, faces)
# N = int(vertices.shape[0] * 2)
N = int(vertices.shape[0] / 2)
point_cloud, pt_to_face = trimesh.sample.sample_surface_even(trimeshMesh, N)
torchPC = torch.tensor(point_cloud, device=torch.device('cuda:0'), dtype=torch.float32)
face_to_all_pts = defaultdict(list)
for pt in range(len(point_cloud)):
    face_to_all_pts[pt_to_face[pt]].append(pt)
p = mp.plot(vertices, faces, colors, return_plot=True, shading={"wireframe": 0.1})
p.add_points(point_cloud, shading={"point_size": 0.1, "point_color": "green"})
plt.show()

only got 1842/2893 samples!


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.4272670…

c:\ProgramData\miniforge3\envs\test_proj\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


In [44]:
def _normalized_grid(width, height, device='cuda'):
    """Returns grid[x,y] -> coordinates for a normalized window.
    
    Args:
        width, height (int): grid resolution
    """

    # These are normalized coordinates
    # i.e. equivalent to 2.0 * (fragCoord / iResolution.xy) - 1.0
    window_x = torch.linspace(-1, 1, steps=width, device=device) * (width / height)
    window_y = torch.linspace(1,- 1, steps=height, device=device)

    coord = torch.stack(torch.meshgrid(window_x, window_y)).permute(1,2,0)
    return coord


def generate_rays(camera_from, camera_to, width, height, mode='persp', fov=90.0, device='cuda'):
    """Vectorized look-at function, returns an array of ray origins and directions
    URL: https://www.scratchapixel.com/lessons/mathematics-physics-for-computer-graphics/lookat-function
    """

    camera_origin = torch.FloatTensor(camera_from).to(device)
    camera_view = F.normalize(torch.FloatTensor(camera_to).to(device) - camera_origin, dim=0)
    camera_right = F.normalize(torch.cross(camera_view, torch.FloatTensor([0,1,0]).to(device)), dim=0)
    camera_up = F.normalize(torch.cross(camera_right, camera_view), dim=0)

    coord = _normalized_grid(width, height, device=device)
    ray_origin = camera_right * coord[...,0,np.newaxis] * np.tan(np.radians(fov/2)) + \
                 camera_up * coord[...,1,np.newaxis] * np.tan(np.radians(fov/2)) + \
                 camera_origin + camera_view
    ray_origin = ray_origin.reshape(-1, 3)
    print(ray_origin)
    ray_offset = camera_view.unsqueeze(0).repeat(ray_origin.shape[0], 1)
    
    if mode == 'ortho': # Orthographic camera
        ray_dir = F.normalize(ray_offset, dim=-1)
    elif mode == 'persp': # Perspective camera
        ray_dir = F.normalize(ray_origin - camera_origin, dim=-1)
        ray_origin = camera_origin.repeat(ray_dir.shape[0], 1)
    else:
        raise ValueError('Invalid camera mode!')


    return ray_origin, ray_dir

In [47]:
# ray_o and ray_d ~ torch.Tensor (width x height, 3)
# represent rays origin and direction vectors
# camera_from=[-.5,2.5,-2.5]
camera_from = [0.,0,1.]
camera_to = [0,0.5,0]
ray_o, ray_d = generate_rays(
                        camera_from=camera_from,
                        camera_to=camera_to,
                        width=1024,
                        height=1024,
                        mode='persp',
                        fov=60,
                        device='cuda')
# np.random.seed(42)
# p = mp.plot(point_cloud, shading={'point_size':0.2, 'point_color':'black'}, return_plot=True)
# p.add_points(ray_o[0].view(1,3).cpu().numpy(), shading={'point_size':0.2, 'point_color':'green'})
# for i in range(10):
#     p.add_lines(ray_o[0].view(1,3).cpu().numpy(), (ray_o[0] + 4.*ray_d[np.random.randint(ray_d.shape[0])]).view(1,3).cpu().numpy(), shading={'line_size':0.2, 'line_color':'blue'})
# plt.show()
print(f'Total of {ray_o.shape[0]} rays generated.')

tensor([[-0.5774,  0.9636,  0.3638],
        [-0.5774,  0.9626,  0.3633],
        [-0.5774,  0.9616,  0.3628],
        ...,
        [ 0.5774, -0.0672, -0.1516],
        [ 0.5774, -0.0682, -0.1521],
        [ 0.5774, -0.0692, -0.1526]], device='cuda:0')
Total of 1048576 rays generated.


In [13]:
width = 1024; height = 1024
window_x = torch.linspace(-1, 1, steps=width, device=device) * (width / height)
window_y = torch.linspace(1,- 1, steps=height, device=device)

coord = torch.stack(torch.meshgrid(window_x, window_y)).permute(1,2,0)

In [43]:
print(torch.stack(torch.meshgrid(window_x, window_y)).permute(1,2,0))

tensor([[-1.0000,  1.0000],
        [-1.0000,  0.9980],
        [-1.0000,  0.9961],
        ...,
        [-1.0000, -0.9961],
        [-1.0000, -0.9980],
        [-1.0000, -1.0000]], device='cuda:0')
